# Trabajo modelo dimensional 
## Willer Cano Narvaez

El siguiente proyecto que se realizara tiene como objetivo realizar un modelo dimensional para una base de datos que tiene información estadistica de los niveles preescolar,básica y media relacionada con indicadores sectoriales por Municipio sin atipicos, desde el año 2011 hasta 2023.Ademas vamos a ultiza otra base de datos llamada Divipola la cual contiene información detallada de colombia, inclye departamentos, municipios, corregimiento y demas información que nos va ser de ayuda al momento de crear las dimensiones.

# Modelo Estrella

Decidí utilizar un modelo estrella porque es uno de los esquemas más simples y eficientes para estructurar datos en proyectos de análisis. Este modelo me permitió organizar la información separando claramente las dimensiones descriptivas (como tiempo, municipio y departamento) de los hechos numéricos que quería analizar, como la tasa de matriculación y la cobertura neta. Al tener una tabla de hechos central conectada a dimensiones limpias y optimizadas, se facilita mucho el uso de consultas en SQL, mejora la comprensión del modelo y hace más rápido el análisis de grandes volúmenes de información educativa.

In [1]:
# ===================================================================
# PASO 1: CONFIGURACIÓN E INSTALACIÓN DE LIBRERÍAS
# ===================================================================

import pandas as pd
import requests
import sqlite3

print("Librerías importadas.")


Librerías importadas.


In [2]:
# ===================================================================
# PASO 2: EXTRACCIÓN BASE PRINCIPAL DESDE API (EDUCACIÓN)
# ===================================================================

import pandas as pd
import requests
import sqlite3

print("Extrayendo datos de educación desde la API del MEN...")

api_url_edu = "https://www.datos.gov.co/resource/nudc-7mev.json?$limit=50000"

try:
    response = requests.get(api_url_edu)
    response.raise_for_status()
    data = response.json()
    df_edu = pd.DataFrame(data)
    print(f"¡Base de educación cargada! {len(df_edu)} filas.")
except requests.exceptions.RequestException as e:
    print(f"Error al extraer educación: {e}")
    df_edu = pd.DataFrame()
except Exception as e:
    print(f"Error inesperado: {e}")
    df_edu = pd.DataFrame()


Extrayendo datos de educación desde la API del MEN...
¡Base de educación cargada! 14585 filas.


In [3]:
df_edu

,a_o,c_digo_municipio,municipio,c_digo_departamento,departamento,c_digo_etc,etc,poblaci_n_5_16,tasa_matriculaci_n_5_16,cobertura_neta,...,reprobaci_n_primaria,reprobaci_n_secundaria,reprobaci_n_media,repitencia,repitencia_transici_n,repitencia_primaria,repitencia_secundaria,repitencia_media,tama_o_promedio_de_grupo,sedes_conectadas_a_internet
0,2023,05004,Abriaquí,05,Antioquia,3758,Antioquia (ETC),503,62.62,62.62,...,1.96,16.51,2.04,9.52,0,10.46,13.76,2.04,NaN,NaN
1,2023,95025,El Retorno,95,Guaviare,3830,Guaviare (ETC),4438,53.27,53.27,...,7.11,9.39,1.75,9.34,6.95,11.84,8.48,3.16,NaN,NaN
2,2023,95200,Miraflores,95,Guaviare,3830,Guaviare (ETC),2014,32.52,32.52,...,6.93,14.13,7.81,8.65,6.67,9.04,10.25,1.54,NaN,NaN
3,2023,97001,Mitú,97,Vaupés,3831,Vaupés (ETC),10986,59.57,59.57,...,4.04,8.33,4.6,16.18,7.75,21.04,13.84,7.18,NaN,NaN
4,2023,97161,Caruru,97,Vaupés,3831,Vaupés (ETC),1228,51.3,51.3,...,7.32,15.28,7.27,9.24,2.86,7.62,14.85,3.64,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14580,2011,5036,Angelópolis,5,Antioquia,3758,Antioquia (ETC),1707,78.85,78.9,...,3.61,9.5,7.32,0.71,0,0.7,1.08,0,19.57,100
14581,2011,5034,Andes,5,Antioquia,3758,Antioquia (ETC),10244,84.45,84.5,...,0.58,0.04,2.69,5.41,0.73,5.53,6.9,4.11,24.43,93.44
14582,2011,5031,Amalfi,5,Antioquia,3758,Antioquia (ETC),5552,97.71,97.7,...,0,0,0,NaN,0.83,NaN,9.93,4.47,20.01,53.45
14583,2011,5030,Amagá,5,Antioquia,3758,Antioquia (ETC),6631,78.65,78.7,...,6.73,14.46,7.45,0.42,0,0.24,0.91,0,25.05,83.33


## Información base de datos

La base de datos cuenta con 14.585 registros y 41 variables, de las cuales 34 son de tipo numérico, 4 son categóricas (texto) y 3 son enteras. La información está organizada por municipio y año, e incluye métricas clave como tasas de matrícula, cobertura neta y bruta, aprobación, reprobación, deserción y repitencia, lo que permite realizar análisis detallados sobre la situación educativa en Colombia.

In [4]:
# ===================================================================
# PASO 3: EXTRACCIÓN BASE DIVIPOLA DESDE API
# ===================================================================

print("Extrayendo datos de DIVIPOLA desde su API...")

api_url_divipola = "https://www.datos.gov.co/resource/gdxc-w37w.json?$limit=2000"

try:
    response = requests.get(api_url_divipola)
    response.raise_for_status()
    data = response.json()
    df_divipola = pd.DataFrame(data)
    print(f"¡Base DIVIPOLA cargada! {len(df_divipola)} filas.")
except requests.exceptions.RequestException as e:
    print(f"Error al extraer DIVIPOLA: {e}")
    df_divipola = pd.DataFrame()
except Exception as e:
    print(f"Error inesperado: {e}")
    df_divipola = pd.DataFrame()


Extrayendo datos de DIVIPOLA desde su API...
¡Base DIVIPOLA cargada! 1122 filas.


In [5]:
df_divipola

,cod_dpto,dpto,cod_mpio,nom_mpio,tipo_municipio,longitud,latitud
0,05,ANTIOQUIA,05001,MEDELLÍN,Municipio,"-75,581775","6,246631"
1,05,ANTIOQUIA,05002,ABEJORRAL,Municipio,"-75,428739","5,789315"
2,05,ANTIOQUIA,05004,ABRIAQUÍ,Municipio,"-76,064304","6,632282"
3,05,ANTIOQUIA,05021,ALEJANDRÍA,Municipio,"-75,141346","6,376061"
4,05,ANTIOQUIA,05030,AMAGÁ,Municipio,"-75,702188","6,038708"
...,...,...,...,...,...,...,...
1117,97,VAUPÉS,97889,YAVARATÉ,Área no municipalizada,"-69,203337","0,609142"
1118,99,VICHADA,99001,PUERTO CARREÑO,Municipio,"-67,487095","6,186636"
1119,99,VICHADA,99524,LA PRIMAVERA,Municipio,"-70,410515","5,486309"
1120,99,VICHADA,99624,SANTA ROSALÍA,Municipio,"-70,859499","5,136393"


## Información base de datos

La base de datos DIVIPOLA contiene información geográfica y administrativa de los municipios de Colombia. Está compuesta por 1,122 registros y 7 columnas, que incluyen datos como el código y nombre del departamento, código y nombre del municipio, tipo de entidad (municipio, isla o área no municipalizada), y coordenadas geográficas (latitud y longitud). En cuanto a los tipos de datos, hay 5 columnas de tipo texto y 2 columnas numéricas que son latitud y longitud.

## Limpieza de la base

In [6]:
# ===================================================================
# PASO 4: LIMPIEZA Y TRANSFORMACIÓN DE LAS BASES EXTRAÍDAS
# ===================================================================

print("Iniciando limpieza de columnas y tipos de datos...")


# Normalizar nombres de columnas
df_edu.columns = df_edu.columns.str.strip().str.lower().str.replace(" ", "_")

# Convertir columnas numéricas (algunas vienen como string desde la API)
columnas_float = ['tasa_matriculación_5_16', 'cobertura_neta']
for col in columnas_float:
    if col in df_edu.columns:
        df_edu[col] = pd.to_numeric(df_edu[col], errors='coerce')

# Convertir año a entero
if 'año' in df_edu.columns:
    df_edu['año'] = pd.to_numeric(df_edu['año'], errors='coerce', downcast='integer')


# Normalizar columnas
df_divipola.columns = df_divipola.columns.str.strip().str.lower().str.replace(" ", "_")

# Renombrar columna conflictiva
if 'tipo:_municipio_/_isla_/_área_no_municipalizada' in df_divipola.columns:
    df_divipola = df_divipola.rename(columns={
        'tipo:_municipio_/_isla_/_área_no_municipalizada': 'tipo_municipio'
    })

# Corregir coordenadas (longitud y latitud)
if 'longitud' in df_divipola.columns:
    df_divipola['longitud'] = df_divipola['longitud'].str.replace(',', '.')
    df_divipola['longitud'] = pd.to_numeric(df_divipola['longitud'], errors='coerce')

if 'latitud' in df_divipola.columns:
    df_divipola['latitud'] = df_divipola['latitud'].str.replace(',', '.')
    df_divipola['latitud'] = pd.to_numeric(df_divipola['latitud'], errors='coerce')

print("Limpieza completada. Listo para modelado dimensional.")


Iniciando limpieza de columnas y tipos de datos...
Limpieza completada. Listo para modelado dimensional.


Durante el proceso de limpieza se estandarizaron los nombres de las columnas poniéndolos en minúscula y reemplazando espacios por guiones bajos para facilitar su uso en consultas SQL. En la base de educación, algunas variables numéricas como la tasa de matriculación y la cobertura venían como texto, por lo que se convirtieron a tipo numérico para poder hacer análisis estadísticos. También se transformó la variable del año a tipo entero. En la base DIVIPOLA se ajustó el nombre de una columna que tenía caracteres especiales y se corrigieron los valores de longitud y latitud, que venían con comas, pasándolos a formato numérico. Esta limpieza es importante para evitar errores al unir las tablas y hacer análisis más precisos.

In [7]:
# ===================================================================
# PASO 5: CREACIÓN DE BASE DE DATOS SQLITE Y GUARDADO DE TABLAS CRUDAS
# ===================================================================

print("Creando base de datos SQLite en memoria...")
conn = sqlite3.connect("../Datos/modelo_educativo.db")
df_edu.to_sql("educacion_raw", conn, if_exists="replace", index=False)
df_divipola.to_sql("divipola", conn, if_exists="replace", index=False)

print("Tablas 'educacion_raw' y 'divipola' guardadas en SQLite.")


Creando base de datos SQLite en memoria...
Tablas 'educacion_raw' y 'divipola' guardadas en SQLite.


In [8]:
print(df_edu.columns.tolist())
print(df_divipola.columns.tolist())


['a_o', 'c_digo_municipio', 'municipio', 'c_digo_departamento', 'departamento', 'c_digo_etc', 'etc', 'poblaci_n_5_16', 'tasa_matriculaci_n_5_16', 'cobertura_neta', 'cobertura_neta_transici_n', 'cobertura_neta_primaria', 'cobertura_neta_secundaria', 'cobertura_neta_media', 'cobertura_bruta', 'cobertura_bruta_transici_n', 'cobertura_bruta_primaria', 'cobertura_bruta_secundaria', 'cobertura_bruta_media', 'deserci_n', 'deserci_n_transici_n', 'deserci_n_primaria', 'deserci_n_secundaria', 'deserci_n_media', 'aprobaci_n', 'aprobaci_n_transici_n', 'aprobaci_n_primaria', 'aprobaci_n_secundaria', 'aprobaci_n_media', 'reprobaci_n', 'reprobaci_n_transici_n', 'reprobaci_n_primaria', 'reprobaci_n_secundaria', 'reprobaci_n_media', 'repitencia', 'repitencia_transici_n', 'repitencia_primaria', 'repitencia_secundaria', 'repitencia_media', 'tama_o_promedio_de_grupo', 'sedes_conectadas_a_internet']
['cod_dpto', 'dpto', 'cod_mpio', 'nom_mpio', 'tipo_municipio', 'longitud', 'latitud']


# Dimensiones

## Dimensión Municipio

In [9]:
# ===================================================================
# PASO 6: CREACIÓN DE LA DIMENSIÓN MUNICIPIO 
# ===================================================================

print("Creando dimensión municipio...")

# Eliminar si ya existe
conn.execute("DROP TABLE IF EXISTS dim_municipio;")

# Crear tabla con los nombres correctos desde la API
query_dim_municipio = """
CREATE TABLE dim_municipio AS
SELECT 
    ROW_NUMBER() OVER () AS id_municipio,
    e.c_digo_municipio AS codigo_municipio,
    e.municipio,
    d.dpto AS nombre_departamento,
    d.tipo_municipio,
    d.longitud,
    d.latitud
FROM educacion_raw e
LEFT JOIN divipola d 
    ON e.c_digo_municipio = d.cod_mpio
GROUP BY e.c_digo_municipio, e.municipio;
"""

conn.execute(query_dim_municipio)

print("Dimensión municipio creada con éxito.")



Creando dimensión municipio...
Dimensión municipio creada con éxito.


La dimensión municipio se crea con el objetivo de estructurar y organizar la información geográfica de cada municipio de forma única y estandarizada dentro del modelo dimensional. Para ello, se realizó una unión entre la base principal de educación y la base DIVIPOLA, aprovechando que esta última contiene datos oficiales y actualizados como el tipo de municipio, longitud y latitud. Esta integración permite enriquecer el análisis con variables territoriales y facilita consultas posteriores al relacionar correctamente los registros educativos con su ubicación geográfica. Además, la dimensión municipio es fundamental para construir relaciones con la tabla de hechos y garantizar una buena relación del modelo.

In [10]:
import pandas as pd
import unicodedata

# Leer la dimensión actual
df_mpio = pd.read_sql("SELECT * FROM dim_municipio", conn)

# Limpiar nombre de municipio
def limpiar(texto):
    texto = str(texto).lower()
    texto = unicodedata.normalize('NFKD', texto).encode('ASCII', 'ignore').decode('utf-8')
    return ''.join(c for c in texto if c.isalnum() or c.isspace()).strip()

df_mpio["municipio_clean"] = df_mpio["municipio"].apply(limpiar)

# Eliminar duplicados por código (nos quedamos con el primero que aparezca)
df_mpio_limpia = (
    df_mpio.sort_values("municipio")
    .drop_duplicates(subset=["codigo_municipio"])
)

# Guardar tabla limpia y definitiva
df_mpio_limpia[["id_municipio", "codigo_municipio", "municipio_clean"]].to_sql(
    "dim_municipio_limpia", conn, if_exists="replace", index=False
)

print(" Dimensión de municipio limpiada y sin duplicados.")

 Dimensión de municipio limpiada y sin duplicados.


Durante el proceso de análisis, pudimos identificar que la dimensión municipio presentaba inconsistencias importantes, ya que algunos municipios aparecían más de una vez debido a variaciones en la forma como estaban escritos, como diferencias en mayúsculas, tildes o caracteres especiales. Esto generaba duplicidad de registros, lo cual afectaba la calidad de los cruces con la tabla de hechos. Por eso, decidimos crear un proceso de limpieza en el que normalizamos los nombres de los municipios (quitando tildes y dejando solo caracteres válidos) y eliminamos los duplicados, conservando un único registro por cada código de municipio. Finalmente, dejamos una nueva versión de esta tabla llamada dim_municipio_limpia, que ya se puede usar sin errores para los análisis.

## Dimensión Departamento

In [11]:
# ===================================================================
# PASO 7: CREACIÓN DE LA DIMENSIÓN DEPARTAMENTO
# ===================================================================

print("Creando dimensión departamento...")

# Eliminar si ya existe
conn.execute("DROP TABLE IF EXISTS dim_departamento;")

# Crear la tabla usando nombres correctos
query_dim_departamento = """
CREATE TABLE dim_departamento AS
SELECT 
    ROW_NUMBER() OVER () AS id_departamento,
    c_digo_departamento AS codigo_departamento,
    departamento
FROM educacion_raw
GROUP BY c_digo_departamento, departamento;
"""

conn.execute(query_dim_departamento)

print("Dimensión departamento creada con éxito.")


Creando dimensión departamento...
Dimensión departamento creada con éxito.


La creación de la dimensión departamento fue fundamental para estructurar adecuadamente el modelo dimensional, ya que nos permite organizar la información educativa según cada entidad territorial. Esta dimensión se construyó a partir de la base principal agrupando por el código y el nombre del departamento para asegurar que no se repitan registros. Al asignar un identificador único a cada uno, logramos establecer una llave primaria que se puede relacionar fácilmente con la tabla de hechos. Esta tabla es clave porque permite responder preguntas agregadas por departamento y facilita análisis comparativos entre regiones.

In [12]:
import pandas as pd
import unicodedata

# Traer la dimensión original
df_dep = pd.read_sql("SELECT * FROM dim_departamento", conn)

# Función para limpiar nombre
def limpiar_nombre(texto):
    texto = str(texto).lower()
    texto = unicodedata.normalize('NFKD', texto).encode('ASCII', 'ignore').decode('utf-8')
    return ''.join(c for c in texto if c.isalnum() or c.isspace()).strip()

# Limpiar y agrupar por codigo_departamento para garantizar unicidad
df_dep["departamento_clean"] = df_dep["departamento"].apply(limpiar_nombre)

# Eliminar duplicados por codigo_departamento (clave segura)
df_dep_limpia = (
    df_dep.sort_values("departamento") 
    .drop_duplicates(subset=["codigo_departamento"])
)

# Guardar nueva dimensión
df_dep_limpia[["id_departamento", "codigo_departamento", "departamento_clean"]].to_sql(
    "dim_departamento_limpia", conn, if_exists="replace", index=False
)

print("Dimensión limpia corregida sin duplicados por código.")


Dimensión limpia corregida sin duplicados por código.


En este paso hicimos lo mismo que con la dimensión municipio, ya que nos dimos cuenta de que la dimensión de departamento también presentaba registros duplicados por errores en los nombres, como uso de tildes o diferencias mínimas de escritura. Para solucionarlo, aplicamos una función que estandariza los nombres, eliminando tildes y caracteres especiales, y luego agrupamos por el código del departamento, que es el identificador más confiable. Así logramos una versión limpia, garantizando que cada departamento esté representado una sola vez, y evitando inconsistencias al momento de construir la tabla de hechos o hacer análisis posteriores.

## Dimensión Tiempo

In [13]:
# ===================================================================
# PASO 8: CREACIÓN DE LA DIMENSIÓN TIEMPO
# ===================================================================

print("Creando dimensión tiempo...")

# Eliminar si ya existe
conn.execute("DROP TABLE IF EXISTS dim_tiempo;")

# Crear la tabla usando el campo 'a_o'
query_dim_tiempo = """
CREATE TABLE dim_tiempo AS
SELECT 
    ROW_NUMBER() OVER () AS id_tiempo,
    a_o AS anio
FROM educacion_raw
GROUP BY a_o
ORDER BY a_o;
"""

conn.execute(query_dim_tiempo)

print("Dimensión tiempo creada con éxito.")


Creando dimensión tiempo...
Dimensión tiempo creada con éxito.


La dimensión de tiempo se construyó a partir del campo de año presente en la base de datos de educación. Esta dimensión es fundamental porque permite analizar cómo varían las métricas educativas a lo largo del tiempo, facilitando comparaciones entre años. Lo que hicimos fue extraer los años únicos, ordenarlos cronológicamente y asignarles un identificador que sirva como clave en la tabla de hechos. Esta estructura ayuda a mantener el modelo organizado y a garantizar que los análisis temporales se puedan hacer de manera eficiente.

# Tabla de Hechos

In [14]:
# ===================================================================
# PASO 9: CREACIÓN DE LA TABLA DE HECHOS
# ===================================================================

conn.execute("DROP TABLE IF EXISTS hechos_educacion;")

query_hechos_final = """
CREATE TABLE hechos_educacion AS
SELECT 
    t.id_tiempo,
    m.id_municipio,
    d.id_departamento,
    e.tasa_matriculaci_n_5_16 AS tasa_matriculacion,
    e.cobertura_neta
FROM educacion_raw e
JOIN dim_tiempo t ON e.a_o = t.anio
JOIN dim_municipio_limpia m ON e.c_digo_municipio = m.codigo_municipio
JOIN dim_departamento_limpia d ON e.c_digo_departamento = d.codigo_departamento;
"""

conn.execute(query_hechos_final)
print("Tabla de hechos reconstruida con municipios y departamentos limpios.")



Tabla de hechos reconstruida con municipios y departamentos limpios.


La tabla de hechos se diseñó con el objetivo de centralizar las variables numéricas más relevantes del análisis educativo, como la tasa de matriculación y la cobertura neta, relacionándolas con las dimensiones de tiempo, municipio y departamento. Para evitar problemas de duplicación o inconsistencias, se vinculó únicamente con las versiones limpias de las dimensiones, asegurando que cada registro esté correctamente referenciado por su código único. Esta estructura nos permite realizar análisis comparativos por territorio y año de manera precisa, manteniendo la integridad del modelo y facilitando consultas eficientes en el esquema estrella.

In [15]:
# ===================================================================
# PASO 10: VALIDACIÓN FINAL DE LA TABLA DE HECHOS
# ===================================================================

print("Mostrando las primeras filas de la tabla de hechos...")

# Leer y mostrar las primeras 5 filas
df_hechos = pd.read_sql("SELECT * FROM hechos_educacion LIMIT 5", conn)
print(df_hechos)


Mostrando las primeras filas de la tabla de hechos...
   id_tiempo  id_municipio  id_departamento tasa_matriculacion  cobertura_neta
0         13             4                2              62.62           62.62
1         13          1261               36              53.27           53.27
2         13          1262               36              32.52           32.52
3         13          1263               37              59.57           59.57
4         13          1264               37               51.3           51.30


En la tabla de hechos podemos ver cómo cada fila representa un registro único que combina un año, un municipio y un departamento con sus respectivos indicadores educativos. Las columnas id_tiempo, id_municipio e id_departamento son claves foráneas que conectan con las dimensiones creadas, permitiendo acceder al contexto geográfico y temporal. Las variables tasa_matriculacion y cobertura_neta son los hechos o métricas numéricas que se desean analizar. Por ejemplo, en la primera fila se indica que en el año correspondiente al id_tiempo 13, el municipio con id_municipio 4 en el departamento 2 tuvo una tasa de matriculación y cobertura neta del 62.62%. Esta estructura facilita el análisis por zonas y periodos, lo que nos va permitis esto es hacer aanalsisi mas precisos y organizados para el entendimiento del usuario.

# Preguntas

Respecto a la población del municipio ¿Que porcentaje de escolaridad hay?

In [16]:
# ===================================================================
# PREGUNTA 1: Porcentaje de escolaridad por municipio
# ===================================================================

query_1 = """
SELECT 
    m.municipio,
    h.tasa_matriculacion
FROM hechos_educacion h
JOIN dim_municipio m ON h.id_municipio = m.id_municipio
ORDER BY h.tasa_matriculacion DESC;
"""

df_p1 = pd.read_sql(query_1, conn)
df_p1.head(10)


,municipio,tasa_matriculacion
0,Achí,99.98
1,Villagarzón,99.98
2,Paipa,99.97
3,Arauca,99.97
4,San Sebastián de Buenavista,99.96
5,Liborina,99.95
6,Palmar de Varela,99.95
7,Tocancipá,99.94
8,El Castillo,99.94
9,Rionegro,99.93


Respondiendo a la pregunta en base a los resultados obtenidos, podemos evidenciar que varios municipios registran porcentajes de escolaridad mayor al 99% lo que nos da a entender que la cobertura esta casi en su total de la población analizada, esto es muy positivo porque lo que nos quiere decir esto es que los niños y jóvenes que deberian estar estudiando si lo estan haciendo, municipios como Achí, Villagarzón, Paipa , Arauca, etc. Son municipios que no son muy nombrados y que uno pensaria que al ser tan apartados o en zonas de conflicto las condiciones de educación no son buenas pero su cobertura es solida, los niños si estan asistiendo a clase. 

¿Cómo compararía el rendimiento educativo por municipios?

In [17]:
# ===================================================================
# PREGUNTA 2: Rendimiento Educativo por Municipio
# ===================================================================

query_2 = """
SELECT 
    m.municipio,
    ROUND(AVG(e.aprobaci_n), 2) AS promedio_aprobacion,
    ROUND(AVG(e.reprobaci_n), 2) AS promedio_reprobacion,
    ROUND(AVG(e.deserci_n), 2) AS promedio_desercion
FROM educacion_raw e
JOIN dim_municipio m ON e.c_digo_municipio = m.codigo_municipio
GROUP BY m.municipio
ORDER BY promedio_aprobacion DESC
LIMIT 10;
"""

df_p2 = pd.read_sql(query_2, conn)
df_p2


,municipio,promedio_aprobacion,promedio_reprobacion,promedio_desercion
0,Puerto Arica,98.97,0.03,1.31
1,Tibirita,98.42,0.43,1.16
2,La Palma,98.28,0.20,1.52
3,Guatavita,98.27,0.27,1.45
4,Miriti - Paraná,98.21,0.35,1.70
5,Chaguaní,98.19,0.16,1.65
6,Pupiales,98.10,0.67,1.23
7,Sutatausa,97.96,0.11,1.93
8,Aguada,97.72,0.90,1.39
9,Junín,97.68,0.98,1.34


Para poder responder a la preguntar del como comparar el rendimiento educativo por municipio, consideré que las variables mas importantes eran el promedio de aprobación, el promedio de reprobación y el promedio de deserción, porque tome estas variables y no otras porque estas nos muestra que tan bien están avanzando los estudiantes, cuántos presentan dificultades y cuántos abandonaron el estudio. Ya analizando los resultados podemos identificar los munucipios como Puerto Arica, Tibirita y La Palma tiene niveles altos de aprobación superiores a un 98% pero que tambien tienen a la vez unosniveles muy bajos en cuanto a la reprobación y deserción, esto nos  hace pensar que en estos municipios los estudiantes si estan pasando de grado y ademas pocos estan dejando a un lado sus estudios, algo muy positivo en el rendimiento educativo esto nos ayuda mucho a ver que municipios estan bien o que municipios necesitan de apoyo si muchos jovenes se estan saliendo puede ser una causa externa o si estan reprobando muchos que es lo que esta pasando en la entidad educativa.

¿Que departamentos son los que mejor cobertura tienen? ¿Pueden hacer cálculo con SQL?

In [18]:
# ===================================================================
# PREGUNTA 3: Departamentos con mejor cobertura
# ===================================================================

query_3 = """
SELECT 
    d.departamento,
    ROUND(AVG(h.cobertura_neta), 2) AS cobertura_promedio
FROM hechos_educacion h
JOIN dim_departamento d ON h.id_departamento = d.id_departamento
GROUP BY d.departamento
ORDER BY cobertura_promedio DESC;
"""

df_p3 = pd.read_sql(query_3, conn)
df_p3


,departamento,cobertura_promedio
0,Quindio,94.58
1,Sucre,93.85
2,Cesar,93.76
3,Magdalena,93.28
4,Meta,90.74
5,Bogotá D.C.,89.49
6,Tolima,89.24
7,Casanare,88.57
8,Cundinamarca,88.56
9,Antioquia,88.10


Para poder respodner la pregutan sobre qué departamentos tienen mejor cobertura educativa, primero me di a la tarea de identificar la variable que me iba ayudar a responder dicha pregunta, la que considere fue cobertura neta ya que esta indica el procentaje de niños y jóvenes en edad escolar que efectivamente están matriculados al nivel educactivo que corresponde. Se realizo el calculo por SQL donde calculé el promedio de cobertura por departamento y los ordené de mayor a menor. Ya hahablando de los resultados obtenidos podemos ver que los mayores niveles de cobertura son Quindio, Sucre, Cesar y Magdalena superiores al 93% lo que nos india que tienen un buen desempeño al acceso educativo para los niños y jovenes. lo que llama la atención es que departamentos como Bogota o Antioquia  no esten de primeras si se supone que tienen mas recursos para invertir a la educación y mas entidades educativas, pero lo que llega a pensar que pasa eso es debido a que departamentos como Quindio, Sucre o Cesar al ser mas pequeñis es mas facil garantizar la educación a todos sus niños en cambio los departamentos grandes tienen factores como es la migración, la desigualdad, sobrepoblacion y saturación en las entidades educativas que hace que afecte la cobertura y muchos niños no tengan ese derecho a la educación esto nos da una conclusión y es que no siempre el tamaño del departamento hace que tenga los mejores resultados educativos.

In [19]:
query_registros_departamento = """
SELECT 
    d.departamento,
    COUNT(*) AS total_registros
FROM hechos_educacion h
JOIN dim_departamento d ON h.id_departamento = d.id_departamento
GROUP BY d.departamento
ORDER BY total_registros DESC;
"""

df_registros_departamento = pd.read_sql(query_registros_departamento, conn)
df_registros_departamento


,departamento,total_registros
0,Antioquia,1625
1,Boyacá,1599
2,Cundinamarca,1508
3,Santander,1131
4,Nariño,832
5,Tolima,611
6,Bolívar,598
7,Valle del Cauca,546
8,Cauca,546
9,Norte de Santander,520


In [20]:
conn.close()
